<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/database/amazon_product_selection_pet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [53.1 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [ ]:
# Create a table to count
product_count = df.groupby("product_id").agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "reviews_count")
product_count.show()

+----------+-------------+
|product_id|reviews_count|
+----------+-------------+
|B0084OHUIO|            8|
|B0012UP46M|           98|
|B00C7CBWUQ|           39|
|B00TQRL9U8|            1|
|B00E0JJ96I|           90|
|B000A8CUSM|          473|
|B00KCKUGI4|           89|
|B006MBRARC|           85|
|B00UICARQM|            1|
|B012GZ2Z2M|            1|
|B003A23HZK|          279|
|B002DZI0RU|          544|
|B00SHXO5VM|           30|
|B008VFXDTG|           42|
|B0002AQBCE|          286|
|B00FFFNIFO|            2|
|B00DW79Y4G|           57|
|B0057XF6U4|           50|
|B00DZVQAA0|           16|
|B00LFNH69E|           15|
+----------+-------------+
only showing top 20 rows



In [ ]:
products_filtered = product_count.filter(product_count.reviews_count > 2000)
products_filtered.show()

+----------+-------------+
|product_id|reviews_count|
+----------+-------------+
|B0037NKDSG|         2242|
|B000RIA95G|         2058|
|B000W5SLB8|         2370|
|B00CX6LJ22|         2137|
|B000LPOUNW|         2222|
|B000MD57OO|         2999|
|B0002D31QU|         2996|
|B000P7JKD6|         2347|
|B0009X29WK|         5277|
|B00063446M|         3500|
|B0021L8W6K|         2036|
|B005E2S77C|         3925|
|B000L3XYZ4|         4562|
|B001LWRFW2|         2408|
|B004QBDO0M|         2267|
|B000FSN0A4|         2732|
|B004X6UEH6|         2425|
|B000634MH8|         3659|
|B0002I0O60|         2498|
|B000084E6V|         2099|
+----------+-------------+
only showing top 20 rows

